In [1]:
import dask
dask.config.config


{'temporary-directory': None,
 'dataframe': {'shuffle-compression': None},
 'array': {'svg': {'size': 120}}}

In [2]:
from dask_jobqueue import LSFCluster

In [3]:
# Per node specification
cluster = LSFCluster(
    scheduler_options={"dashboard_address": ":3762"},
    cores=8,
    processes=1, # default sqrt(cores). set to one to max threads per machine.  better for numpy.  see https://docs.dask.org/en/latest/setup/single-machine.html
    memory="4 GB",
    project="VEN201",
    walltime="00:30",
    job_extra=["-nnodes 1"],          # <--- new!
    header_skip=["-R", "-n ", "-M"],  # <--- new!
    interface='ib0',
    use_stdin=False
)

## Lets See what is sent to LSF

In [4]:
print(cluster.job_script())

#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -P VEN201
#BSUB -W 00:30
#BSUB -nnodes 1

/ccs/home/vanstee/.conda/envs/powerai-ornl/bin/python -m distributed.cli.dask_worker tcp://10.41.0.35:46463 --nthreads 8 --memory-limit 4.00GB --name name --nanny --death-timeout 60 --interface ib0



In [5]:
from dask.distributed import Client
client = Client(cluster)

In [6]:
# client.restart()

In [7]:
client

Client Scheduler: tcp://10.41.0.35:46463 Dashboard: http://10.41.0.35:3762/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [9]:
cluster.scale(2)

In [ ]:
client

In [19]:
!bjobs

JOBID   USER       STAT   SLOTS    QUEUE       START_TIME    FINISH_TIME   JOB_NAME                      
357170  vanstee    RUN    43       batch       Sep 21 14:30  Sep 21 15:00  dask-worker                   
357171  vanstee    RUN    43       batch       Sep 21 14:31  Sep 21 15:01  dask-worker                   
357196  vanstee    RUN    43       batch       Sep 21 14:34  Sep 21 15:04  dask-worker                   
357197  vanstee    RUN    43       batch       Sep 21 14:34  Sep 21 15:04  dask-worker                   
357198  vanstee    PEND      -     batch             -             -       dask-worker                   
357199  vanstee    PEND      -     batch             -             -       dask-worker                   
357200  vanstee    PEND      -     batch             -             -       dask-worker                   
357201  vanstee    PEND      -     batch             -             -       dask-worker                   
357202  vanstee    PEND      -     batch      

# Numpy simple example ...

In [13]:
import dask.array as da
# 2.5 B element array , 500 chunks
x = da.random.random([50000,50000], chunks=[2500,2500])


In [18]:
cluster.scale(8)

In [41]:
x = x.persist()
x

dask.array<random_sample, shape=(50000, 50000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

In [46]:
y = x.T ** x - x.mean()

In [47]:
y.persist()

dask.array<sub, shape=(50000, 50000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>

In [38]:
#del(y)
y.compute()

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
# Persist vs Compute https://distributed.dask.org/en/latest/memory.html
# use compute when the return value is small and you want to feed result into other analyses.
# use persist (similar to cache in spark) to trigger computation and pin results to memory.  
# Follow actions build task graphs, but only up to this point as it will use the value calculated by persist.